# Evaluation and Finetuning of Summarization Models in Okareo

<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/classification_finetuning_eval_part1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 🎯 Goals

After using this notebook, you will be able to:
- Finetune an open source LLM for summarization
- Evaluate the finetuned model in Okareo

## Problem Statement: Summarization

Suppose we are developing a meeting transcript summarization application. To do this, we will prompt an LLM to take a meeting transcript (several hundred to a thousand words) and to generate a concise summary.

We will start by evaluating a zero-shot summarization LLM (GPT-4o-mini). We will then attempt to improve the latency and performance by fine-tuning an open source model for our summarization task.

This notebook focuses on finetuning an open source LLM, [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct), for summarization purposes.

## Upload the Data as a Scenario in Okareo

First, we setup our Okareo client. You will need API token from [https://app.okareo.com/](https://app.okareo.com/). (Note: You will need to sign up and generate an API token.)

In [ ]:
# get Okareo client

from okareo import Okareo

OKAREO_API_KEY = "<YOUR_OKAREO_API_KEY>"
okareo = Okareo(OKAREO_API_KEY)

In [ ]:
# get all meeting summaries
import pandas as pd
meeting_transcripts = "../../demos/.okareo/flows/meetings_long.jsonl"
json_df = pd.read_json(meeting_transcripts, lines=True)

json_df.head()

In [ ]:
# pick transcripts that are shorter than 2048

N = 2048
sampled_df = json_df[json_df['source'].apply(lambda x: len(x) < 2048)].reset_index(drop=True)
print(f"# summaries shorter than {N} characters: {sampled_df.shape[0]}")
sampled_df = sampled_df.loc[:,['summary', 'source']]
sampled_df = sampled_df.rename(columns={'source': 'input', 'summary': 'result'})
sampled_df.head()

In [ ]:
# get the max length (in both characters/words) of the reference summaries
n_words_max = sampled_df['result'].apply(lambda x: len(x.split(" "))).max()
n_chars_max = sampled_df['result'].apply(lambda x: len(x)).max()
n_words_mean = sampled_df['result'].apply(lambda x: len(x.split(" "))).mean()
n_chars_mean = sampled_df['result'].apply(lambda x: len(x)).mean()
n_words_median = sampled_df['result'].apply(lambda x: len(x.split(" "))).median()
n_chars_median = sampled_df['result'].apply(lambda x: len(x)).median()
print(f"Max words: {n_words_max} | Max characters: {n_chars_max}")
print(f"Mean words: {n_words_mean} | Mean characters: {n_chars_mean}")
print(f"Median words: {n_words_median} | Median characters: {n_chars_median}")

Let's see if we can get an LLM to summarize the meeting transcripts in 350 words or fewer (i.e., around the median character length in the transcript dataframe).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sampled_df['input'], sampled_df['result'], test_size=0.2)

In [ ]:
from okareo_api_client.models.scenario_set_create import ScenarioSetCreate
from okareo_api_client.models.seed_data import SeedData

# Create a scenario set of the WebBizz documents

split_names = ["train", "test"]
input_splits = [X_train, X_test]
result_splits = [y_train, y_test]
split_ids = {}

for split_name, input_split, result_split in zip(split_names, input_splits, result_splits):
    seed_data = []
    for article, summary in zip(input_split, result_split):
        seed_data.append(SeedData(input_=article, result=summary))

    summary_scenario = okareo.create_scenario_set(
        ScenarioSetCreate(
            name=f"Meeting Summaries ({split_name})", seed_data=seed_data
        )
    )
    split_ids[split_name] = summary_scenario.scenario_id
    print(summary_scenario.app_link)

In [ ]:
TRAIN_SPLIT_ID = "0081b0e2-225a-4d2b-b73f-8af46561da6c"
TEST_SPLIT_ID = "bf732090-785c-42e4-9eb9-25ccf7d51794"

In [ ]:
# in addition to pre-defined Okareo checks, we will use a few custom CodeBasedChecks to evaluate the model

from checks.character_count import Check

okareo.create_or_update_check(
    name="character_count",
    description="The number of characters in the model_output.",
    check=Check(),
)

from checks.word_count import Check

okareo.create_or_update_check(
    name="word_count",
    description="The number of words in the model_output.",
    check=Check(),
)

from checks.is_character_count_under_350 import Check

okareo.create_or_update_check(
    name="under_350_characters",
    description="Whether or not the number of characters in the model_output is under 350.",
    check=Check(),
)

Evaluate on the test split. We will use the train split later in this notebook.

In [ ]:
from datetime import datetime
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models import TestRunType

OPENAI_API_KEY="<YOUR_OPENAI_API_KEY>"

with open('prompts/zero_shot_summarization.txt', "r") as f:
    zero_shot_prompt = f.read()

USER_PROMPT_TEMPLATE = "Article: {scenario_input}"

model_name = "GPT-4o-mini (Zero-Shot Summarization)"
mut = okareo.register_model(
    name=model_name,
    model=OpenAIModel(
        model_id="gpt-4o-mini",
        temperature=0,
        system_prompt_template=zero_shot_prompt,
        user_prompt_template=USER_PROMPT_TEMPLATE,
    ),
    update=True,
)

test_run_name = f"Summarization Run (test split)"
run_resp = mut.run_test(
    name=test_run_name,
    scenario=TEST_SPLIT_ID,#split_ids['test'],
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    checks=[
        "latency",
        "fluency_summary",
        "character_count",
        "word_count",
        "under_350_characters",
    ],
)

In [ ]:
run_resp.app_link

### Fine-tune Phi-3.5-mini-instruct for Summarization

With our goal of reducing the length of summaries in mind, let's fine-tune an open source LLM for summarization.

In [ ]:
# get the data points
sdp = okareo.get_scenario_data_points(TRAIN_SPLIT_ID)

In [ ]:
finetuning_data = [{'input': dp.input_, 'result': dp.result} for dp in sdp]

In [ ]:
finetuning_data[0]

### Configure Phi-3 for finetuning

Now we set up a finetuning run on [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) using the finetuning instruction scenario.

Setup is mostly boilerplate from [here](https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/sample_finetune.py).

In [ ]:
import sys

import datasets
from peft import LoraConfig
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

###################
# Hyper-parameters
###################
training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 5.0e-05,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 3,
    "max_steps": -1,
    "output_dir": "./finetuned_phi3", # checkpoint directory
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    }

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Param efficient finetuning config
peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    # "target_modules": "all-linear",
    "modules_to_save": None,
}
train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

In [ ]:
import logging

###############
# Setup logging
###############
logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process a small summary
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

In [ ]:
################
# Model Loading
################

checkpoint_path = "microsoft/Phi-3.5-mini-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    quantization_config=bnb_config,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 3072
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

In [ ]:
##################
# Data Processing
##################

with open('prompts/finetune_summarization.txt', "r") as f:
    SHORT_SYSTEM_PROMPT_TEMPLATE = f.read()

def apply_chat_template(
    example,
    tokenizer,
):
    messages = example["messages"]
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False)
    return example

In [ ]:
from datasets import Dataset
message_formatted_data = []
for data in finetuning_data:
    messages = [
        {'role': 'user', 'content': f"{SHORT_SYSTEM_PROMPT_TEMPLATE}\n\nArticle: {data['input']}"},
        {'role': 'assistant', 'content': data['result']},
    ]
    message_formatted_data.append({'messages': messages})

dataset = Dataset.from_list(message_formatted_data)

column_names = list(dataset.features)

processed_dataset = dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to train_sft",
)


In [ ]:
###########
# Training
###########
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=processed_dataset,
    # eval_dataset=processed_test_dataset,
    max_seq_length=3072,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True
)
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()



In [ ]:
# ############
# # Save model
# ############
trainer.save_model(train_conf.output_dir)

Go on to Part 2 to run the evaluation of the fine-tuned model!

In [ ]:
print(f"TEST_SPLIT_ID={TEST_SPLIT_ID}")